In [8]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [9]:
from langchain_neo4j import Neo4jGraph

graph = Neo4jGraph(
    url = os.getenv("NEO4J_URI"),
    username = os.getenv("NEO4J_USERNAME"),
    password = os.getenv("NEO4J_PASSWORD")
)

In [10]:
#테스트 쿼리 실행 - Neo4j 연결이 제대로 작동하는지 확인하기 위한 간단한 테스트
#Cypher 쿼리를 사용하여 Test 레이블을 가진 노드를 생성하고 반환한다.
cypher_query = """
CREATE (n:Test {name: "Hello AuraDB"})
RETURN n
"""

#이 쿼리가 성공적으로 실해오디면 Neo4J DB연결이 정상적으로 작동하는 것임
graph.query(cypher_query)

[{'n': {'name': 'Hello AuraDB'}}]

In [4]:
def reset_database(graph):
    """데이터베이스 초기화하기"""
    #모든 노드와 관계 삭제
    graph.query("MATCH (n) DETACH DELETE n")

    #모든 제약조건 삭제
    constraints = graph.query("SHOW CONSTRAINTS")
    for constraint in constraints:
        constraint_name = constraint.get("name")
        if constraint_name:
            graph.query(f"DROP CONSTRAINT {constraint_name}")
    
    #모든 인덱스 삭제
    indexes = graph.query("SHOW INDEXES")
    for index in indexes:
        index_name = index.get("name")
        index_type = index.get("type")
        if index_name and index_type != "CONSTRAINT":
            graph.query(f"DROP INDEX {index_name}")
    
    print("데이터베이스가 초기화되었습니다.")

#데이터베이스 초기화
reset_database(graph)

데이터베이스가 초기화되었습니다.


In [11]:
#Neo4j 데이터베이스에 노드 생성하기 (CREATE 구문 사용)
#동일한 노드가 이미 존재해도 중복생성되므로 주의 해야 함
cypher_query = """
// 주석은 아래와 같이 처리할 수 있음
// 노드 레이블 : Person (사람을 나타내는 레이블)
// 노드 속성 정의:
// - name : "서소망" (이름)
// - age : 10 (나이)
// - email : somang@gmail.com (이메일 주소)
// p는 생성된 노드를 참조하는 변수명입니다.

CREATE (p:Person {name : "서소망", age: 10, email : "somang@gmail.com"}) 
"""

graph.query(cypher_query)

[]

### 1. 노드(Node) 표현

In [7]:
#CREATE 구문을 사용하여 여러 노드를 한번에 생성할 수 있음
cypher_query = """
// 노드 레이블 정의 : 
// - Person : 사람을 나타내는 레이블
// - City : 도시를 타나내는 레이블
//
// Person 노드 속성 정의
// - name : 사람의 이름 (문자열)
// - age : 사람의 나이 (숫자)
//
// City 노드 속성 정의 :
// - name : 도시 이름 (문자열)
// - population : 인구수 (숫자)
//
// 변수 a,b,c는 각각 생성된 노드를 참조하는 변수명 입니다.
CREATE
 (a: Person {name: "김철수", age :25}),
 (b: Person {name: "이영희", age :28}),
 (c: City {name : "서울", population: 9700000})
"""

#graph.query() 를 사용하여 Cypher 쿼리를 실행합니다.
#이 명령은 Neo4j 데이터베이스에 세 개의 노드를 한 번에 생성합니다.
graph.query(cypher_query)

[]

### 2. 관계 (Relationship) 표현

In [11]:
#기존 노드 사이에 관계 생성하기
cypher_query="""
// 관계 레이블 : KNOWS (알고 있다는 관계를 나타냄)
// 관계 속성 : since (관계가 시작된 시점을 타나내는 속성)
// 관계 방향 : a -> b (서소망이 김철수를 알고 있는 방향성)
// 관계 속성 값 : since : 2020 (2020년부터 관계가 시작됨)
// 관계 생성 의미 : 서소망은 김철수를 2020년부터 알게 됨

// MATCH 구문으로 두 개의 기존 노드를 찾음
MATCH
    (a: Person {name: "서소망"}), // Person 레이블을 가진 이름이 서소망인 노드를 찾아 변수 a에 할당
    (b: Person {name: "김철수"}) // Person 레이블을 가진 이름이 김철수인 노드를 찾아 변수 b에 할당

//CREATE 구문으로 두 노드 사이에 방향성 있는 관계 생성
CREATE (a)-[r:KNOWS {since: 2020}] -> (b) // a에서 b로 향하는 KNOWS 타입의 관계 r 생성, since 속성 추가

//RETURN 구문으로 노드와 관계 정보 반환
RETURN a,b,r // 생성된 관계와 연결되 두 정보를 함께 반환
"""

graph.query(cypher_query)

[{'a': {'name': '서소망', 'age': 10, 'email': 'somang@gmail.com'},
  'b': {'name': '김철수', 'age': 25},
  'r': ({'name': '서소망', 'age': 10, 'email': 'somang@gmail.com'},
   'KNOWS',
   {'name': '김철수', 'age': 25})}]

In [ ]:
# 노드와 관계를 한번에 생성하는 Cypher 쿼리
cypher_query = """
// 노드 레이블 : Person, City (각각 사람과 도시를 나타내는 레이블)
// CREATE 구문으로 노드와 관계를 동시에 생성
CREATE
    // a 변수에 Person 레이블을 가진 노드 생성, 속성으로 name = "이삼백" 설정
    (a: Person {name: "이삼백"})-[:LIVES_IN]-> 
    
    // c 변수에 City 레이블을 가진 노드 생성, 속성으로 name ="도쿄" 설정
    // LIVES_IN 관계는 '살고 있다'는 의미로 이삼백이 도쿄에 거주함을 나타냄
    (c: City {name: "도쿄"})
// RETURN 구문으로 생성된 두 노드 정보 반환
RETURN a,c
"""

graph.query(cypher_query)

[{'a': {'name': '이삼백'}, 'c': {'name': '도쿄'}}]

In [19]:
# 기존에 생성된 노드들 사이에 관계를 추가하는 Cypher 쿼리
cypher_query="""
// 관계 레이블 : LIVES_In (거주한다는 의미의 관계 타입)
// 관계 방향 : a -> c (이영희가 서울에 산다는 방향성)
// 관계 속성 : since : 2020 (2020년부터 관계가 시작됨)
// 관계 생성 의미 : 이영희는 서울에 2020년부터 거주하기 시작함

// MATCH 구문으로 두 개의 기존 노드를 찾음
MATCH 
    (a: Person {name: "이영희"}),
    (c: City {name: "서울"})
    
// CREATE 구문으로 두 노드 사이에 방향성 있는 관계 생성
CREATE (a)- [r:LIVES_IN {since: 2020}] -> (c) // a에서 c로 향하는 LIVES_IN 타입의 관계 r 생성하고 since 속성 추가

// RETURN 구문으로 노드의 관계 정보 반환
RETURN a,c,r //생성된 관계와 연결된 두 노드 정보를 함께 반환
"""


graph.query(cypher_query)

[{'a': {'name': '이영희', 'age': 28},
  'c': {'name': '서울', 'population': 9700000},
  'r': ({'name': '이영희', 'age': 28},
   'LIVES_IN',
   {'name': '서울', 'population': 9700000})}]

In [24]:
# 기존에 생성된 노드들 사이에 여러 관계를 추가하는 Cypher 쿼리
cypher_query = """
// 첫 번째 관계 정보
// 관계 레이블 : LIVES_IN (거주한다는 의미의 관계 타입)
// 관계 방향 : a -> c (서소망이 서울에 산다는 방향성)
// 관계 속성 : since 2010 (2010년부터 관계가 시작됨)
// 관계 생성 의미 : 서소망은 서울에 2010년부터 거주하기 시작함

// 두 번째 관계 정보
// 관계 레이블 : KNOWS (안다/알고 있다는 의미의 관계 타입)
// 관계 방향 : a -> b (서소망이 이삼백을 안다는 방향성)
// 관계 속성 : since 2020 (2020년부터 관계가 시작됨)
// 관계 생성 의미 : 서소망은 이삼백을 2020년부터 알게 됨

// 세 번째 관계 정보
// 관계 레이블 : KNOWS (안다/알고 있다는 의미의 관계 타입)
// 관계 방향 : b -> a (이삼백이 서소망을 안다는 방향성)
// 관계 속성 : since 2020 (2020년부터 관계가 시작됨)
// 관계 생성 의미 : 서소망은 이삼백을 2020년부터 알게 됨
// 두 번째와 세 번째 관계를 통해 양방향 관계(b <-> a)를 표현함

// MATCH 구문으로 세 개의 기존 노드를 찾음
MATCH
    (a: Person {name: "서소망"}), // Person 레이블을 가진 이름이 서소망인 노드를 찾아 변수 a에 할당
    (b: Person {name: "이삼백"}),
    (c: City {name : "서울"})

CREATE (a)-[r1:LIVES_IN {since:2010}]->(c)
CREATE (a)-[r2:KNOWS {since:2020}]->(b)
CREATE (b)-[r3:KNOWS {since:2020}]->(a)
"""

graph.query(cypher_query)

[]

In [26]:
cypher_query = """
MATCH (p)-[r:LIVES_IN]->(c)
WITH p, c, collect(r) AS rels
WHERE size(rels) > 1
WITH tail(rels) AS relsToDelete
UNWIND relsToDelete AS rel
DELETE rel
RETURN size(relsToDelete) AS DeletedRelationshipsCount;
"""

graph.query(cypher_query)

[{'DeletedRelationshipsCount': 1},
 {'DeletedRelationshipsCount': 2},
 {'DeletedRelationshipsCount': 2}]

### 3. 변수 (Variable) / 별칭 (Alias) 활용

In [27]:
# 노드 조회 (RETURN 구문)
cypher_query = """
// Person 레이블을 가진 모든 노드 조회
MATCH (p:Person)
RETURN p
"""
result = graph.query(cypher_query)

print("노드 조회 결과:")
for record in result :
    print(record)

노드 조회 결과:
{'p': {'name': '김철수', 'age': 25}}
{'p': {'name': '서소망', 'age': 10, 'email': 'somang@gmail.com'}}
{'p': {'name': '이영희', 'age': 28}}
{'p': {'name': '이삼백'}}


In [28]:
#노드 조회 (속성반환)
cypher_query = """
// Person 레이블을 가진 모든 노드의 name 속성 조회
MATCH(p: Person)
RETURN p.name AS Name
"""

result = graph.query(cypher_query)
print("노드 조회 결과:")
for record in result:
    print(record)

노드 조회 결과:
{'Name': '김철수'}
{'Name': '서소망'}
{'Name': '이영희'}
{'Name': '이삼백'}


### 4. 주요구문 (MATCH, WHERE, SET, DELETE, REMOVE 등)

In [ ]:
# Neo4j 그래프DB에서 노드 가느이 관계를 조회하는 예제
cypher_query="""
// Person 레이블을 가진 노드와 City 레이블을 가진 노드 사이의 관계 조회
// (p:Person) : Person 타입의 노드를 p 변수에 할당
// -[r:LIVES_IN]->:LIVES_IN 타입의 관계를 r 변수에 할당하고, 방향은 Person에서 City로 향함
// (c: City) : City 타입의 노드를 c 변수에 할당
MATCH (p:Person)-[r:LIVES_IN]->(c:City)

// p.name As Name : Person 노드의 name 속성을 'Name'이라는 별칭으로 반환
// c.name As City : City 노드의 name 속성을 'City' 라는 별칭으로 반환
RETURN p.name As Name, c.name As City
"""

graph.query(cypher_query)

[{'Name': '서소망', 'City': '서울'},
 {'Name': '이영희', 'City': '서울'},
 {'Name': '이삼백', 'City': '도쿄'}]

In [31]:
# 조건 지정 (WHERE 구문)
# WHERE 구문은 MATCH로 찾은 패턴에 추가 조건을 적용할 때 사용한다.
cypher_query="""
// Person 레이블을 가진 노드 중에서 age 속성이 30 이상이고 사는 도시가 서울인 노드 조회
MATCH (p:Person)-[r:LIVES_IN]->(c:City)

WHERE p.age >=20 AND c.name="서울"
RETURN p
"""

graph.query(cypher_query)

[{'p': {'name': '이영희', 'age': 28}}]

In [32]:
#노드 속성 업데이트 (SET 구문 사용)
cypher_query="""
MATCH (p:Person {name:"이삼백"})

// p로 참조된 노드의 'age' 속성 값을 30 으로 설정합니다.
// 해당 속성이 이미 존재하면 값을 변경하고, 없으면 새로 생성한다.
SET p.age =30

// 업데이트된 노드를 결과로 반환합니다.
RETURN p
"""

graph.query(cypher_query)

[{'p': {'name': '이삼백', 'age': 30}}]

In [34]:
#노드 삭제 작업 수행
cypher_query="""
MATCH (p:Person {name:"이영희"})

//주의 : 이 방식은 관계가 없는 노드만 삭제 가능합니다.
//노드에 관계가 있는 경우 ConstraintValidationFailed 오류 발생
//관계가 있는 노드를 삭제하려면 DETACH DELETE를 사용해야 합니다.
DELETE p
"""

graph.query(cypher_query)

ConstraintError: {code: Neo.ClientError.Schema.ConstraintValidationFailed} {message: Cannot delete node<2>, because it still has relationships. To delete this node, you must first delete its relationships.}

In [ ]:
# 관계가 있는 노드 삭제 (DETACH DELETE 사용)
cypher_query="""
MATCH (p:Person {name: "이영희"})
DETACH DELETE p
"""

# 결과로 이영희 노드와 그에 연결된 모든 관계가 함께 삭제됩니다.
graph.query(cypher_query)

[]

In [36]:
# 관계 삭제 수행
cypher_query="""
// Person 레이블을 가진 노드 중에서 name 속성이 "서소망"인 노드와 "김철수"인 노드 사이의 관계를 찾습니다.
MATCH
    (a: Person {name:"서소망"})-[r]->(b:Person {name:"김철수"})

// DELETE r 명령어를 사용하여 찾은 관계만 삭제합니다.
// 이 명령은 노드는 그대로 유지하고 두 노드 사이의 관계만 제거합니다.
// 노드 삭제와 달리 관계 삭제는 DETACH 키워드가 필요하지 않습니다.
DELETE r
"""

graph.query(cypher_query)

[]

In [37]:
# 속성 제거 (REMOVE 구문 사용)
cypher_query="""
MATCH (p:Person {name: "서소망"})

// REMOVE 명령어를 사용하여 찾은 노드의 age 속성을 제거한다.
// 이 명령은 노드 자체나 다른 속성은 그대로 유지하고 지정된 속성만 삭제합니다.
// 속성이 존재하지 않는 경우에도 오류 없이 실행됩니다.
REMOVE p.age

RETURN p
"""

graph.query(cypher_query)

[{'p': {'name': '서소망', 'email': 'somang@gmail.com'}}]

In [39]:
# 레이블 제거 (REMOVE 구문 사용)
cypher_query="""
MATCH (p:Person {name:"서소망"})

// REMOVE 명령어를 사용하여 찾은 노드에서 Person 레이블을 제거합니다.
// 이 명령은 노드 자체나 속성은 그대로 유지하고 지정된 레이블만 삭제합니다.
// 레이블이 제거되면 해당 노드는 더 이상 Person으로 분류되지 않습니다.
// 노드에 다른 레이블이 있다면 그 레이블은 유지됩니다.
REMOVE p:Person

RETURN p
"""
graph.query(cypher_query)

[{'p': {'name': '서소망', 'email': 'somang@gmail.com'}}]

In [40]:
# 레이블 추가 및 속성 설정 (SET 구문 사용)
cypher_query="""
// name 속성이 "서소망"인 노드를 찾습니다.
// 이 노드는 이전 쿼리에서 Person 레이블이 제거되었습니다.
MATCH (p {name: "서소망"})

// SET 명령어를 사용하여 노드에 Person 레이블을 다시 추가하고 age 속성을 10으로 설정합니다.
SET p: Person, p.age =10

RETURN p
"""

graph.query(cypher_query)

[{'p': {'name': '서소망', 'age': 10, 'email': 'somang@gmail.com'}}]

In [ ]:
# 나이 순 정렬 - ORDER BY 와 LIMIT 사용 예제
cypher_query="""
// DB에서 Person으로 분류된 모든 개체를 검색합니다.
MATCH (p:Person)

RETURN p

// 나이를 오름차순으로 정렬 (최대 2명까지)
ORDER BY p.page ASC
LIMIT 2
"""
result = graph.query(cypher_query)

print("나이 순 정렬 결과:")
for record in result:
    print(record)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: page)} {position: line: 7, column: 12, offset: 75} for query: '\n// DB에서 Person으로 분류된 모든 개체를 검색합니다.\nMATCH (p:Person)\n\nRETURN p\n\nORDER BY p.page ASC\nLIMIT 2\n'


나이 순 정렬 결과:
{'p': {'name': '서소망', 'age': 10, 'email': 'somang@gmail.com'}}
{'p': {'name': '김철수', 'age': 25}}


In [43]:
# 나이 순 정렬 (SKIP 사용)
cypher_query = """
MATCH (p:Person)
RETURN p
ORDER BY p.age ASC

// SKIP 절을 사용하여 결과의 처음 1개를 건너뜁니다.
SKIP 1

LIMIT 1
"""

result = graph.query(cypher_query)
print("나이 순 정렬 결과:")
for record in result:
    print(record)

나이 순 정렬 결과:
{'p': {'name': '김철수', 'age': 25}}
